<a href="https://colab.research.google.com/github/haidour18/Benchemarking-Recommender-systems-/blob/main/Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle
!pip install -q keras
!pip install numpy
!pip install sklearn
!pip install scipy
!pip install surprise
!pip install apyori
!pip install matrix_factorization
!pip install factor-analyzer
!pip install python-helper

In [3]:
#All the header files required for the code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import helper
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob , csv
import time, joblib
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
from zipfile import ZipFile as zip
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

from IPython.display import display
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import cosine_similarity as cos
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_score
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files 
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change permissions 
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d rounakbanik/the-movies-dataset


 93% 211M/228M [00:04<00:00, 42.7MB/s]
100% 228M/228M [00:04<00:00, 52.2MB/s]


In [8]:
# Dézippier le .zip obtenu
from zipfile import ZipFile
file_name = 'the-movies-dataset.zip'
with ZipFile (file_name ,'r') as zip : 
  zip.extractall()
  print('Done')

Done


In [6]:
movies=pd.read_csv("movies_metadata.csv")
ratings=pd.read_csv("ratings.csv")
movies=movies.iloc[:3000,[3,5,8]]
movies.head(5)
movies.dropna()
ratings.dropna()
movies.head(5)

,genres,id,original_title
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,Toy Story
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,Jumanji
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,Grumpier Old Men
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,Waiting to Exhale
4,"[{'id': 35, 'name': 'Comedy'}]",11862,Father of the Bride Part II


In [7]:
Nmovies=movies.shape[0]

In [8]:
import ast

# this function extracts words out of dictionaries and formats them
def word_extractor(row_of_words):
    words_joined = []
    if (type(row_of_words)!=str or type(ast.literal_eval(row_of_words))!=list):
        words_joined = ['']
    else:
    # extract words from the dictionaries
        word_list = ast.literal_eval(row_of_words)
        for w in range(0,len(word_list)):
            word_list[w] = word_list[w]['name']
            word_list[w] = word_list[w].replace(" ","")
        words_joined.append(' '.join(word_list))    
    return words_joined

# applying the word_extractor function to dict. features
genres = []
for m in range(0,Nmovies):
    genres.append(word_extractor(movies['genres'][m]))
    movies['genres'][m]=word_extractor(movies['genres'][m])


In [9]:
movies.head(5)


,genres,id,original_title
0,[Animation Comedy Family],862,Toy Story
1,[Adventure Fantasy Family],8844,Jumanji
2,[Romance Comedy],15602,Grumpier Old Men
3,[Comedy Drama Romance],31357,Waiting to Exhale
4,[Comedy],11862,Father of the Bride Part II


In [10]:
ratings.rename(columns={'movieId': 'id'},inplace=True)


In [11]:
ratings.head(5)

,userId,id,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [12]:
data=ratings.join(movies,on="id", how='left', lsuffix='_left', rsuffix='_right')
data=data.dropna()
data.tail(35)

,userId,id_left,rating,timestamp,genres,id_right,original_title
26024133,270896,1917,3.0,1257033804,[Mystery Horror ScienceFiction],9424,Disturbing Behavior
26024134,270896,1923,3.0,1257031845,[Comedy Family],20173,The Barefoot Executive
26024135,270896,1961,5.0,1257031604,[Drama Music Romance],42121,Tender Mercies
26024136,270896,1968,4.5,1257031883,[Adventure],35144,The Journey of Natty Gann
26024137,270896,2000,3.0,1257034164,[Comedy],6471,The Jerk
26024138,270896,2001,3.0,1257034458,[Comedy Mystery],9442,Dead Men Don't Wear Plaid
26024139,270896,2003,3.0,1257031234,[Drama],13697,Grand Canyon
26024140,270896,2028,5.0,1257031870,[Animation Drama Family],15171,Charlotte's Web
26024141,270896,2054,3.0,1257034154,[Comedy Horror ScienceFiction],2182,Attack of the Killer Tomatoes!
26024142,270896,2100,3.0,1257031179,[Drama Crime],2762,Young and Innocent


In [5]:

# Data Processing
# Converting Genres into different columns 
# Here we just create columns and put there initial value as 0

x = data.genres
a = []
for i in x:
    abc = i
    a.append(abc)
a = pd.DataFrame(a)   
b = a[0].unique()
for i in b:
    data[i] = 0
data.head(15)

NameError: ignored